In [1]:
import pipeline
import pandas as pd
#import importlib
#importlib.reload(pipeline)

In [2]:
filepath = '../data/covid_df_protocol_3.pkl'
weeks_for_cv = [0,1,2,3,4,5,6]
df = pd.read_pickle(filepath)
n_weeks_prediction = 2
results = {}
for i in weeks_for_cv:
    shorter_df = pipeline.cut_df_on_weeks(df, i)
    df_train, df_test = pipeline.split_and_scale_on_last_weeks(shorter_df, n_weeks_prediction)
    df_train = df_train.dropna()
    df_test = df_test.dropna()
    X_train, y_train = pipeline.divide_target_and_features(df_train, 'Confirmed Cases')
    X_test, y_test = pipeline.divide_target_and_features(df_test, 'Confirmed Cases')
    dct = pipeline.train_and_evaluate(X_train, y_train, X_test, y_test)
    results['Cutting weeks: ' +str(i)] = dct


Cutting dataframe on date: 2020-05-30


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 16189573943419.486, tolerance: 5045545789.8601675
  positive)


Lasso: Features with highest magnitude                                coefficients in absolute value
                              Feature    Coefficient
175  Country_United States of America  275845.235851
18                    StringencyIndex -171920.236787
100                     Country_Italy   70731.222679
157                     Country_Spain   59716.377736
21               EconomicSupportIndex  -37589.983309
86                    Country_Germany   34828.091077
59                     Country_Canada  -33914.314329
170                    Country_Turkey   28607.303279
2                C2_Workplace closing   25848.837024
9                   E1_Income support   24832.356850

Bias: 1244054865.62
Mean squared error: 9103270011.32
RSS: 20127329995037.01
Variance score: 0.36

R2 score: 0.38

Ridge: Features with highest magnitude                                coefficients in absolute value
                              Feature   Coefficient
63                      Country_China -70639.79

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11412059276319.973, tolerance: 3439423263.544699
  positive)


Lasso: Features with highest magnitude                                coefficients in absolute value
                              Feature    Coefficient
177  Country_United States of America  219680.516348
18                    StringencyIndex -145079.850493
102                     Country_Italy   61060.329497
159                     Country_Spain   52493.672100
21               EconomicSupportIndex  -32418.085606
87                    Country_Germany   31500.426126
59                     Country_Canada  -28530.362281
172                    Country_Turkey   22951.700319
2                C2_Workplace closing   21735.420990
9                   E1_Income support   21648.810785

Bias: 933825431.18
Mean squared error: 7927407157.60
RSS: 23163883714501.86
Variance score: 0.34

R2 score: 0.34

Ridge: Features with highest magnitude                                coefficients in absolute value
                              Feature   Coefficient
63                      Country_China -58176.827

...

Lasso: Features with highest magnitude                                coefficients in absolute value
                              Feature   Coefficient
177  Country_United States of America  83632.384011
18                    StringencyIndex -58001.974585
102                     Country_Italy  33192.205043
159                     Country_Spain  27828.772926
21               EconomicSupportIndex -20268.991079
87                    Country_Germany  19180.797952
9                   E1_Income support  13144.753527
10            E2_Debt/contract relief  12308.868878
98                       Country_Iran  11356.512428
59                     Country_Canada -11293.341463

Bias: 246132076.37
Mean squared error: 3889015622.95
RSS: 11686491946959.87
Variance score: 0.25

R2 score: 0.22

Ridge: Features with highest magnitude                                coefficients in absolute value
                              Feature   Coefficient
102                     Country_Italy  26664.956768
159    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1100235735051.1118, tolerance: 283986849.9777367
  positive)


Lasso: Features with highest magnitude                                coefficients in absolute value
                              Feature   Coefficient
177  Country_United States of America  46380.848068
18                    StringencyIndex -37312.867666
102                     Country_Italy  23794.670807
159                     Country_Spain  18808.336277
21               EconomicSupportIndex -17796.609923
87                    Country_Germany  12887.910206
10            E2_Debt/contract relief  10971.838841
9                   E1_Income support  10962.703259
98                       Country_Iran   9515.923437
59                     Country_Canada  -7430.341270

Bias: 119124614.25
Mean squared error: 2668965999.42
RSS: 8044263522248.79
Variance score: 0.23

R2 score: 0.17

Ridge: Features with highest magnitude                                coefficients in absolute value
                     Feature   Coefficient
102            Country_Italy  20021.110264
21      EconomicSupportInd

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 399799333315.97205, tolerance: 98227055.74670902
  positive)


Lasso: Features with highest magnitude                                coefficients in absolute value
                              Feature   Coefficient
18                    StringencyIndex -19220.474607
177  Country_United States of America  17501.788852
102                     Country_Italy  15198.476677
21               EconomicSupportIndex -11408.882562
159                     Country_Spain  10793.124110
10            E2_Debt/contract relief   7233.534287
87                    Country_Germany   6881.050517
9                   E1_Income support   6810.412889
98                       Country_Iran   6628.104561
50     Country_Bosnia and Herzegovina  -4056.488762

Bias: 47129171.01
Mean squared error: 1593658686.88
RSS: 4824004845170.68
Variance score: 0.19

R2 score: 0.12

Ridge: Features with highest magnitude                                coefficients in absolute value
                     Feature   Coefficient
102            Country_Italy  13516.036085
21      EconomicSupportInde

Checking variables with highest percentage of NAs in training set:

In [7]:

for i in results:
    print(results[i])

{'Lasso': (1244054865.6249437, 9103270011.323841, 20127329995037.01, 0.35878312906285426), 'Ridge': (1243933075.8605053, 9099119653.565084, 20118153554032.402, 0.3588459025415995), 'Linear Regression': (1239738021.6125278, 9030390579.850908, 19966193572050.36, 0.36100813801257975)}
{'Lasso': (933825431.1752739, 7927407157.598172, 23163883714501.86, 0.3351079750364059), 'Ridge': (933716507.9267613, 7920282483.269298, 23143065416112.887, 0.335185529359571), 'Linear Regression': (931804277.6736747, 7879137911.047895, 23022840976081.95, 0.3365470543327892)}
{'Lasso': (656757891.4123936, 6443985725.935974, 19454392906600.707, 0.310257663474789), 'Ridge': (656674187.4572976, 6436867862.433257, 19432904076686.004, 0.3103455713055425), 'Linear Regression': (655420170.3391175, 6427854646.186622, 19405693176837.41, 0.3116625691040402)}
{'Lasso': (425659026.7938818, 5202577823.271312, 15623341203283.748, 0.28364288387626724), 'Ridge': (425606766.8644897, 5196172848.367119, 15604107063646.457, 0.2